# Objective

The goal of this notebook is to derive the **recursive Bayes Filter**, which allows a robot to **estimate its state over time** given:

- control inputs $u_{1:t}$ (e.g., wheel commands)  
- sensor measurements $z_{1:t}$ (e.g., LiDAR, camera)

We aim to:

1. Represent the robot's belief recursively $\mathrm{bel}(x_t) = fn(bel(x_{t-1}),u_{1:t},z_{1:t})$.  
2. Split the update into **prediction** and **measurement update** steps.  


   
---

# 1. Prerequisites: Bayes' Rule and Notation

## Bayes' Rule
$$
P(A \mid B) = \frac{P(B \mid A)\,P(A)}{P(B)}
$$

## Bayes' Rule with Context  
In robotics, past information is treated as context \(C\):
$$
P(A \mid B, C)
= \frac{P(B \mid A, C)\,P(A \mid C)}{P(B \mid C)}
$$

## Sequence Notation
A sequence is written compactly as:
$$
z_{1:t} = \{z_1, z_2, \ldots, z_t\}
$$

---

# 2. Goal: The Recursive Bayes Filter

Belief is defined as:
$$
\text{bel}(x_t) = p(x_t \mid z_{1:t}, u_{1:t})
$$

Since we aim to describe the belief over time using it's previous beliefs, it might be a good idea to split the probability based on time step (t) and time step (t-1)

$$
p(x_t \mid z_{1:t}, u_{1:t})
= p(x_t \mid z_t, z_{1:t-1}, u_{1:t})
$$

---

# 3. Applying Bayes' Rule

Let  
- \(A = x_t\)  
- \(B = z_t\)  
- \(C = \{z_{1:t-1}, u_{1:t}\}\)

Applying Bayes:
$$
p(x_t \mid z_t, z_{1:t-1}, u_{1:t})
=
\frac{
p(z_t \mid x_t, z_{1:t-1}, u_{1:t})
\;
p(x_t \mid z_{1:t-1}, u_{1:t})
}{
p(z_t \mid z_{1:t-1}, u_{1:t})
}
$$

### Normalizer
$$
\eta = \frac{1}{p(z_t \mid z_{1:t-1}, u_{1:t})}
$$

So:
$$
\text{bel}(x_t)
=
\eta \;
p(z_t \mid x_t, z_{1:t-1}, u_{1:t})
\;
p(x_t \mid z_{1:t-1}, u_{1:t})
$$

---

# 4. The Markov Assumption

Sensors depend only on the current state:
$$
p(z_t \mid x_t, z_{1:t-1}, u_{1:t})
\approx
p(z_t \mid x_t)
$$

Substitute:
$$
\text{bel}(x_t)
=
\eta \;
\underbrace{p(z_t \mid x_t)}_{\text{Measurement Model}}
\;
\underbrace{p(x_t \mid z_{1:t-1}, u_{1:t})}_{\text{Prediction (Prior)}}
$$

---

# 5. Understanding the Prediction Term

We approximate:
$$
p(x_t \mid z_{1:t-1}, u_{1:t})
\approx
p(x_t \mid z_{1:t-1}, u_{1:t-1})
$$

Thus:
$$
\text{bel}(x_t)
=
\eta \;
p(z_t \mid x_t)
\;
p(x_t \mid z_{1:t-1}, u_{1:t-1})
$$

---

# 6. From Prior at \(t\) to Posterior at \(t-1\)

Using the Law of Total Probability:
$$
p(x_t \mid z_{1:t-1}, u_{1:t-1})
=
\int
\underbrace{p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t-1})}_{\text{Transition Probability}}
\;
\underbrace{p(x_{t-1} \mid z_{1:t-1}, u_{1:t-1})}_{\text{Previous Belief}}
\; dx_{t-1}
$$

Replace the prior with the belief:
$$
p(x_t \mid z_{1:t-1}, u_{1:t-1})
=
\int
\underbrace{p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t-1})}_{\text{Transition Probability}}
\;
\underbrace{\text{bel}(x_{t-1})}_{\text{Previous Belief}}
\; dx_{t-1}
$$

---

# 7. Full Recursive Bayes Filter

$$
\text{bel}(x_t)
=
\eta \;
\underbrace{p(z_t \mid x_t)}_{\text{Measurement Model}}
\int
\underbrace{p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t-1})}_{\text{Transition Probability}}
\;
\underbrace{\text{bel}(x_{t-1})}_{\text{Previous Belief}}
\; dx_{t-1}
$$

---

# 8. Simplified Form

Since the transition probability does not depend on measurements, but only the previous pose and the control signls:
$$
p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t-1})
\approx
p(x_t \mid x_{t-1}, u_{t-1})
$$

Final Bayes Filter equation:
$$
\text{bel}(x_t)
=
\eta \;
\underbrace{p(z_t \mid x_t)}_{\text{Measurement Model}}
\int
\underbrace{p(x_t \mid x_{t-1}, u_{t-1})}_{\text{Transition Probability}}
\;
\underbrace{\text{bel}(x_{t-1})}_{\text{Previous Belief}}
\; dx_{t-1}
$$


## How can we solve this recursive framework ? 

The Bayes Filter is fundamentally a **recursive state estimation technique**.  
Different implementations of this recursive formulation depend on the properties of:

- **The transition (motion) model**
- **The measurement (sensor) model**

Key characteristics that influence the choice of implementation include:

- **Linear vs. Non-linear motion models**  
- **Gaussian vs. Non-Gaussian noise distributions**  
- **Parametric vs. Non-parametric representations**

All these characteristics determine how we can practically solve the general Bayes Filter equation while avoding the insane computational complexity of the exact solution.  
Different algorithms are adopted based on these properties **(e.g., Kalman Filter, EKF, UKF, Particle Filter)**.
 

# Motion Models

Since that motion models influnce the practical solution of general bayes filter, we will focus first on the motion models, before moving to the different algorithms used to implement bayes framework practically

